## Custom Transformers

In [1]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

### Define Custom Transformer to remove non alphanumeric charaters

In [2]:
class CleanText(Transformer, HasInputCol, HasOutputCol):
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(CleanText, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)
        
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, dataset):
        def f(text):
            text = re.sub("\W", " ", text)
            text = re.sub(" +", " ", text)
            
            text = re.sub(r"’", "\'", text)
            text = re.sub(r'([^\x00-\x7f])',r'', text)
            return text
        
        t = StringType()
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

### Define Custom Transformer to remove HTML

In [3]:
from bs4 import BeautifulSoup

class BeutifulSoupTagRemover(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(BeutifulSoupTagRemover, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, dataset):

        def f(s):
            cleaned_post = BeautifulSoup(s).text
            return cleaned_post

        t = StringType()
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [4]:
import pyspark
from pyspark.sql.functions import lower, col

cleanText  = CleanText(inputCol="name", outputCol="name1")
beutifulSoupTagRemover = BeutifulSoupTagRemover(inputCol="name1", outputCol="name2")
tokenizer = pyspark.ml.feature.Tokenizer(inputCol="name2", outputCol="name3")
stopWordsRemover = pyspark.ml.feature.StopWordsRemover(inputCol="name3", outputCol="name_preprocessed")

preprocessingPipeline = pyspark.ml.Pipeline(stages=[
    cleanText,
    beutifulSoupTagRemover,
    tokenizer,
    stopWordsRemover
])

In [5]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import Row

savedModel = pipelineModel = PipelineModel.load("hdfs:///data/exercise/LogRegCrossvalModel")

In [6]:
## +-------------------+-----------+-----+
## |      category_code|category_id|count|
## +-------------------+-----------+-----+
## |         boys-shoes|       1599|15400| 1
## |        girls-shoes|       1612|21632| 0
## | mens-lace-up-shoes|       1564|12353| 2
## |     shoes-athletic|       1976| 4899| 4
## |mens-shoes-athletic|       1561| 7935| 3
## |      evening-shoes|       1773|  901| 5
## |       bridal-shoes|       1386|  848| 6
## +-------------------+-----------+-----+

# {u'category_id_indexed': [u'1612',
#   u'1599',
#   u'1564',
#   u'1561',
#   u'1976',
#   u'1773',
#   u'1386']}

In [7]:
mapping = {
    0: 'girls-shoes', 
    1: 'boys-shoes', 
    2: 'mens-lace-up-shoes', 
    3: 'mens-shoes-athletic', 
    4: 'shoes-athletic', 
    5: 'evening-shoes', 
    6: 'bridal-shoes'
}

In [8]:
def translate(mapping):
    def translate_(col):
        return mapping.get(col)
    return udf(translate_, StringType())

In [9]:
def get_prediction(tweet_text):
    try:
        # filter the tweets whose length is greater than 0
        tweet_text = tweet_text.filter(lambda x: len(x['text']) > 0)
        rowRdd = tweet_text.map(lambda w: Row(name=w['text'], created_at=w['created_at']))
        wordsDataFrame = spark.createDataFrame(rowRdd)
        wordsDataFrame = preprocessingPipeline.fit(wordsDataFrame).transform(wordsDataFrame)
        prediction = savedModel.transform(wordsDataFrame)
        prediction_selection = prediction.select('created_at', 'name','prediction')
        prediction_selection = prediction_selection.withColumn("prediction_text", translate(mapping)("prediction"))
        prediction_selection.show(truncate = False)
        prediction_selection.write.mode('append').parquet("hdfs:///data/exercise/shoes_predictions.parquet")
        prediction_selection.write.mode('append').csv("hdfs:///data/exercise/shoes_predictions.csv")
    except ValueError as er:
        pass
    except Exception as er: 
        print(er, str(er))

In [ ]:
import sys, os, json

from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

from pyspark import SparkContext, SparkConf

from pyspark import SparkContext
sc = SparkContext.getOrCreate();
## Create Streaming context , with 30 second interval 
ssc = StreamingContext(sc,  1) 


kafkaProps = {
    "bootstrap.servers": "localhost:9092",
    "group.id": "offers-consumer",
    "fetch.message.max.bytes": "15728640",
    "auto.offset.reset": 'largest'
}

# Stream receiver 
#  Listens to the kafka topic that we just created, so it is getting stream of alerts json data 
kafkaStream = KafkaUtils.createDirectStream(ssc, ['offers'], kafkaProps)
parsed = kafkaStream.map(lambda v: json.loads(v[1]))
# parsed = parsed.map(lambda x: x['text'])
parsed.foreachRDD(get_prediction)

source_dstream = parsed.map(lambda tweet: tweet)

ssc.start()
ssc.awaitTermination()

+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|created_at                    |name                                                                                                                                        |prediction|prediction_text   |
+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|Mon Jul 27 16:05:53 +0000 2020|RT @Nike: Shoes, hoodies, shorts, and more to get you ready for tip-off and beyond.

Shop Now: https://t.co/l7uSdDRtUQ https://t.co/Hme67Kn…|2.0       |mens-lace-up-shoes|
+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+

+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+---------------+
|created_at                    |name                                                                                                                                        |prediction|prediction_text|
+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+---------------+
|Mon Jul 27 16:09:56 +0000 2020|shopppp 🥳🥳🥳🥳🥳🥳🥳                                                                                                                      |0.0       |girls-shoes    |
|Mon Jul 27 16:09:56 +0000 2020|RT @ArleasMommy: @HBCUfessions On top of all the loans you boutta apply for, piss in his shoes. Put Nair in his shampoo. Put chicken and sh…|4.0       |shoes-athletic |
+-

+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+---------------+
|created_at                    |name                                                                                                                                        |prediction|prediction_text|
+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+---------------+
|Mon Jul 27 16:13:55 +0000 2020|RT @NWCfursuits: @FursonaPins How about some fursuit spray? They smell great and disinfect. Great for other things too like shoes, bags etc…|0.0       |girls-shoes    |
+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+------------

+------------------------------+-----------------------------------------------------+----------+---------------+
|created_at                    |name                                                 |prediction|prediction_text|
+------------------------------+-----------------------------------------------------+----------+---------------+
|Mon Jul 27 16:18:16 +0000 2020|Shopping is not fun if you’re wearing the wrong shoes|0.0       |girls-shoes    |
+------------------------------+-----------------------------------------------------+----------+---------------+

+------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|created_at                    |name                                                                                                                                       |prediction|prediction_text   |
+----------------------

+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|created_at                    |name                                                                                                                                        |prediction|prediction_text   |
+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|Mon Jul 27 16:24:34 +0000 2020|Isn't it great? How beautiful is that? #dress #shoes #heels #purse #jewelry #shopping #shoponline #newarrivals https://t.co/Z1KUad5Xdd      |2.0       |mens-lace-up-shoes|
|Mon Jul 27 16:24:34 +0000 2020|@wonderlxost Aww jisung is richest among them. A fan said "rie ya jisung said you are the richest". Rie said no jus… https://t.co/kZwfrXRrdi|0.0       |

...

+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|created_at                    |name                                                                                                                                        |prediction|prediction_text   |
+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|Mon Jul 27 22:50:24 +0000 2020|RT @GetEshoes: Quality Dress shoes
N25,500
40-46
More discounts on https://t.co/sLmow9qewD  WhatsApp: https://t.co/Rymlvqk25r Shop now! Nat…|2.0       |mens-lace-up-shoes|
+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+

+------------------------------+---------------------------+----------+---------------+
|created_at                    |name                       |prediction|prediction_text|
+------------------------------+---------------------------+----------+---------------+
|Mon Jul 27 22:51:37 +0000 2020|I had to sell my joints lol|0.0       |girls-shoes    |
+------------------------------+---------------------------+----------+---------------+

+------------------------------+---------------+----------+------------------+
|created_at                    |name           |prediction|prediction_text   |
+------------------------------+---------------+----------+------------------+
|Mon Jul 27 22:51:38 +0000 2020|Taking notes 😌|2.0       |mens-lace-up-shoes|
+------------------------------+---------------+----------+------------------+

+------------------------------+------------------------------------------------------------------------------------------------------------------------------------

+------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|created_at                    |name                                                                                                                                       |prediction|prediction_text   |
+------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|Mon Jul 27 22:56:01 +0000 2020|RT @___Kwinn: Good day everyone, I sell beautiful bags and shoes at a very affordable price. Trust me you’ll love them a whole lot. Please…|2.0       |mens-lace-up-shoes|
+------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+----------+-----

+------------------------------+-----------------------------+----------+---------------+
|created_at                    |name                         |prediction|prediction_text|
+------------------------------+-----------------------------+----------+---------------+
|Mon Jul 27 23:00:26 +0000 2020|You're irrelevant. God knows.|1.0       |boys-shoes     |
+------------------------------+-----------------------------+----------+---------------+

+------------------------------+----------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|created_at                    |name                                                                                                                        |prediction|prediction_text   |
+------------------------------+----------------------------------------------------------------------------------------------------------------------------+----------+-----

+------------------------------+----+----------+---------------+
|created_at                    |name|prediction|prediction_text|
+------------------------------+----+----------+---------------+
|Mon Jul 27 23:07:44 +0000 2020|Ok  |0.0       |girls-shoes    |
+------------------------------+----+----------+---------------+

+------------------------------+-----------------------------+----------+------------------+
|created_at                    |name                         |prediction|prediction_text   |
+------------------------------+-----------------------------+----------+------------------+
|Mon Jul 27 23:08:34 +0000 2020|Made in China by slave labor!|2.0       |mens-lace-up-shoes|
+------------------------------+-----------------------------+----------+------------------+

+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+-------------------+
|cr